In [12]:
# train_model.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout


In [13]:
# Load data
data = pd.read_csv("C:/Users/DELL/OneDrive/Desktop/Project/data/train.csv")

In [17]:
# Define lists to store images and labels
l1=[]
l2=[] 
for img,ClassId,EncodedPixels in tqdm(data.values):
    image=cv2.imread("C:/Users/DELL/OneDrive/Desktop/Project/data/train_images/{}".format(img),cv2.IMREAD_COLOR)
    image=cv2.resize(image,(120,120))
    l1.append(image)
    l2.append(ClassId)

100%|██████████| 7095/7095 [01:39<00:00, 71.19it/s] 


In [18]:
#Label encoding the input features and one hot encoding the target feature.
encoder = LabelEncoder()

X= np.array(l1)
X = X/255

y = encoder.fit_transform(l2)
y = to_categorical(y)

In [19]:
#Splitting the data into 80% train set and 20% test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,shuffle=True)

In [20]:
# Define the model architecture
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(120,120,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256,activation="relu"))
model.add(Dense(4,activation="softmax"))

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [21]:
#Early Stopping is a form of regularization used to avoid overfitting when training a learner with an iterative method, such as gradient descent. Early stopping rules provide guidance as to how many iterations can be run before the learner begins to over-fit.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,min_delta=0.001,restore_best_weights=True)

In [22]:
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=["accuracy"])

In [23]:
# Train the model
history = model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=128,
                    verbose=1, callbacks=[early_stopping])

Epoch 1/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 41s 700ms/step - accuracy: 0.7027 - loss: 0.9914 - val_accuracy: 0.7259 - val_loss: 0.8242
Epoch 2/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 38s 837ms/step - accuracy: 0.7362 - loss: 0.7936 - val_accuracy: 0.7259 - val_loss: 0.7695
Epoch 3/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 799ms/step - accuracy: 0.7342 - loss: 0.7612 - val_accuracy: 0.7470 - val_loss: 0.7294
Epoch 4/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 32s 710ms/step - accuracy: 0.7478 - loss: 0.7189 - val_accuracy: 0.7477 - val_loss: 0.6684
Epoch 5/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 729ms/step - accuracy: 0.7378 - loss: 0.7026 - val_accuracy: 0.7597 - val_loss: 0.6524
Epoch 6/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 795ms/step - accuracy: 0.7401 - loss: 0.6753 - val_accuracy: 0.7562 - val_loss: 0.6095
Epoch 7/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 738ms/step - accuracy: 0.7590 - loss: 0.6226 - val_accuracy: 0.7512 - val_loss: 0.6075
Epoch 8/15
45/45 ━━━━━━━━━━━━━━━━━━━━ 34s 750ms/step - accuracy: 0.7472 - loss: 0.6377 - val_accu

In [24]:
# Save the trained model
model.save("C:/Users/DELL/OneDrive/Desktop/Project/models/trained_model.h5")